# Pandas

The `numpy` library is excellent for numerical computations, but it lacks support to handle missing data or non-omogeneous arrays. The `pandas` library is based on numpy and extends the numpy functionality, and is currently one of the most widely used tools for data manipulation, providing high-performance, easy-to-use data structures and advanced data analysis tools.

In particular `pandas` features:

* A fast and efficient `DataFrame` object for data manipulation with integrated indexing;
* Tools for reading and writing data between in-memory data structures and different formats (CSV, Excel, SQL, HDF5);
* Convenient label-based slicing, fancy indexing, and subsetting of large data sets;
* Hierarchical axis indexing provides an intuitive way of working with high-dimensional data in a lower-dimensional data structure;
* Smart data alignment and integrated handling of missing data;
* Aggregating and transforming data with a powerful "group-by" engine; 
* High performance merging and joining of data sets;
* Time series-functionalities;
* Highly optimized for performance, with critical code paths written in Cython or C.


In [ ]:
import pandas as pd # standard naming convention
import numpy as np

## Series

Pandas Series represent an extension of the numpy 1D arrays. A Series is equivalent to a numpy array, but the axis  is labeled, and there is the possibility to store heterogeneous data. Labels doesn't need to be unique but must be a hashable type.

One of the most important examples are the time-series, which are used to keep track of the time evolution of a certain quantity.

Link to the official Pandas Series [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html).

In [ ]:
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']

# Calling the Series constructor
# Constructor requires the data, and optionally the indices and data type
sr = pd.Series(np.arange(10)*0.5, index=tuple(letters[:10]), dtype=float)
print("series:\n", sr, '\n')
print("indices:\n", sr.index, '\n')
print("values:", sr.values, type(sr.values), '\n') # values of the Series are actually a numpy array
print("type:\n", sr.dtype, '\n')

print("element by index     :", sr['f'], '\n') # Accessing elements like arrays
print("element by attribute :", sr.f, '\n') # Accessing elements like attributes - not recommended
subsr = sr[['d', 'f', 'h']] # note the double square brackets
print("series subset:\n", subsr, type(subsr), '\n') # Multiple indexing returns another series

In [ ]:
# Extracting elements and operations are the same as numpy array
print(sr[:3], '\n')
print(sr[7:], '\n')
print(sr[::3], '\n')
print(sr[sr > 3], '\n')
print(np.exp(sr), '\n')
print(np.mean(sr), np.std(sr), '\n')

Series may contain non-omogeneous data; in this case, the data type is referred to as `object`. Non-homogeneous data is normally handeled also by pandas and does not represent a problem, however this pays the price of less time-efficient operations.

In [ ]:
# Series can be created from a python dictionary, too
# Note that the elements can be of different types
d = {'b' : 1, 'a' : 'cat', 'c' : [2, 3]}
so = pd.Series(d)
print(so, '\n')


A key difference between pandas Series and numpy arrays is that operations between Series **automatically align the data based on the label**.

Thus, you can write operations without considering whether the Series involved have the same labels, or even the same size.

In [ ]:
s = pd.Series(np.arange(5), index=tuple(letters[:5]))
print("series:\n", s, '\n')
s1 = s[1:] + s
print("shifted sum:\n", s1, '\n')

s2 = s[1:] + s[:-1]
print("double shifted sum:\n", s2, '\n')

### Time series

Time series are very often used to describe the behaviour of a quantity as a function of time. Pandas has a special type of index for that, `DatetimeIndex`, that can be created e.g. with the function `pd.data_range()`.

In [ ]:
# to define a date, the datetime module is very useful
import datetime as dt

date = dt.date.today()
print("Today's date:", date)

# specify year, month, day, hour, minutes, seconds, and us
date = dt.datetime(2020, 11, 12, 10, 45, 10, 15)
print("Date and time:", date)

# otherwise, several notations can also be used
date = 'Nov 9 2020'
# or alternatively
date = '9/11/2020 14:45:00'
print("Date format:", date)

# create DatetimeIndex using ranges:
days = pd.date_range(date, periods=7, freq='D')
print("7 days range:", days)

seconds = pd.date_range(date, periods=3600, freq='s')
print("1 hour in seconds:", seconds)

To learn more about the frequency strings, please check the [documentation](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases).

Timestamped data is the most basic type of time series data that associates values with points in time.

Functions like `pd.to_datetime` can be used to convert between different formats and, for instance, when reading the time stored as a string from a dataset:

In [ ]:
# Get the timestamp, which is the nanoseconds from January 1st 1970
tstamp = pd.Timestamp(date)
#tstamp = pd.Timestamp(dt.datetime(1970, 1, 1, 0, 0, 0, 1))
print("Timestamp:", tstamp.value)

# when creating a timestamp the format can be explicitly passed
ts = pd.to_datetime('2010/11/12', format='%Y/%m/%d')
print("Time:", ts, ", timestamp:", ts.value, ", type:", type(ts))

ts = pd.to_datetime('12-11-2010 10:39', format='%d-%m-%Y %H:%M')
print("Time:", ts, ", timestamp:", ts.value, ", type:", type(ts))

A standard series can be created, and (a range of) elements can be used as indices:

In [ ]:
print("index:\n", days, '\n')
tseries = pd.Series(np.arange(len(days)), index=days)
print("time series:\n", days, '\n')
# Extracting elements
print("slice by position:\n", tseries[0:4], '\n')
print("slice by date range:\n", tseries['2020-9-11' : '2020-9-14'], '\n') # note that includes end time

`pd.to_datetime` can also be used to create a `DatetimeIndex` if the argument is a list:

In [ ]:
print(pd.to_datetime([1, 2, 3, 4], unit='D', origin=pd.Timestamp('1980-02-03')))

## DataFrame

A pandas DataFrame can be thought as a tabular spreadsheet, although the performance, the functionalities and the capabilities are very different.

Similarly to Series, the DataFrame structure also contains labeled axes (rows and columns). Arithmetic operations **align on both row and column labels**. Each column in a DataFrame is a Series object: as a matter of fact, a DataFrame can be thought of as a dict-like container for Series objects.

The elements can be of all types, and missing data could be present too (represented as NaN).

For future reference (or for people already familiar with R), a pandas DataFrame is also similar to the R DataFrame.

Link to the official [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).

### DataFrame constructor

A DataFrame objects can be created by passing a dictionary of objects. Note that the dictionary values are not omogeneous and do not have the same length. In these cases, pandas will automatically adjust the sizes, by replicating the content or adding NaN if necessary.

In [ ]:
df = pd.DataFrame({
    'A' : 1.,
    'B' : pd.Timestamp('20130102'),
    'C' : pd.Series(3, index=range(4), dtype='float32'),
    'D' : np.arange(7, 11),
    'E' : pd.Categorical(["test", "train", "test", "train"]),
})
df

An example of DataFrame with a DatatimeIndex object as index:

In [ ]:
entries = 10
columns = ['A', 'B', 'C', 'D']
dates = pd.date_range('11/9/2020 14:45:00', freq='h', periods=entries) # days/month/year
df = pd.DataFrame(np.arange(entries*4).reshape(entries, len(columns)), index=dates, columns=columns)
df # pay attention that the date is printed as year-day-month

### Viewing Data

In [ ]:
df.head()

In [ ]:
df.tail(4)

In [ ]:
df.index

In [ ]:
df.columns

In [ ]:
df.values

In [ ]:
df.describe()

In [ ]:
df.T

In [ ]:
df.sort_index(axis=1, ascending=False)

In [ ]:
df.sort_values(by="C", ascending=False)

## Selection

### Slicing

DataFrame slicing allows to select a subset of the DataFrame, or an entire column (a Series):

In [ ]:
## standard and safe
print(df['A'], '\n', type(df['A']), '\n') # Returns a Series (a column)

## equivalent but dangerous (imagine blank spaces in the name of the column, or a column named "T")
print(df.A, '\n')

In [ ]:
# selecting rows by position. Returns another DataFrame (a copy)
print(df[0:3])

# or by index range
print(df["2020-11-09 14:45:00" : "2020-11-09 16:45:00"])

### Selection by label

The most common way to select elements, rows, or columns in a DataFrame is by using the `.loc[]` method.

`.loc` supports multi-indexing, and returns a **copy** of the DataFrame.

In [ ]:
# getting a part of the DataFrame (in this case, a row)) using a label. Returns a Series
dfs = df.loc[dates[0]]
print(dfs, '\n', type(dfs), '\n')

In [ ]:
# selecting on a multi-axis by label:
dfa = df.loc[:, ['A','B']]
print("Are df and dfa the same object?", np.may_share_memory(df, dfa))
dfa

In [ ]:
# showing label slicing, both endpoints are included:
df.loc['2020-11-09 18:45:00':'2020-11-09 20:45:00', ['A','B']]

In [ ]:
# getting an individual element
print(df.loc[dates[1], 'A'], '\n', type(df.loc[dates[1], 'A']), '\n')

The `.at()` method is equivalent to `.loc[]`. Use `at` if you only need to get or set a single value in a DataFrame or Series.

In [ ]:
print(df.at[dates[1], 'A'])

### Selecting by position

`.iloc[]` is similar ot `.loc[]`, but instead of labels, it uses pure integer-location based indexing for selection by position.

But differently from `.loc[]`, `.iloc[]` returns a **view**, not a copy.

Yes, views and copies are ambiguous in DataFrames. And it gets worse! This appears to be a long standing issue for pandas. Read a nice article on this topic [here](https://www.practicaldatascience.org/html/views_and_copies_in_pandas.html).

In [ ]:
# select via the position of the passed integers:
print(df.iloc[3], '\n')

# row and column ranges selected with numpy-like notation:
dfv = df.iloc[3:5, 0:2]
print(dfv, '\n')

print("Are df and dfv the same object?", np.may_share_memory(df, dfv))

In [ ]:
# selecting rows 1,2 and 4 for columns 0 and 2
df.iloc[[1, 2, 4], [0, 2]]

In [ ]:
# slicing rows explicitly
df.iloc[1:3, :]

# slicing columns explicitly
df.iloc[:, 1:3]

Similary to `.loc[]` and `.at[]`, there is also `.iat[]` alongside `.iloc[]`:

In [ ]:
# selecting an individual element by position: no difference between iloc and iat
print(df.iloc[1,1], ", type:", type(df.iloc[1,1]))
print(df.iat[1,1], ", type:", type(df.iat[1,1]))

### Masks

Boolean masks can be used in the same way as numpy, and they represent a very powerful way of filtering out data with certain features. Just like numpy fancy indexing, using a mask returns a **copy** of the DataFrame.

In [ ]:
# Selecting on the basis of boolean conditions applied to the whole DataFrame
dfc = df[df > 10]
dfc.iat[0, 0] = -999
# a DataFrame with the same shape is returned, with NaN's where condition is not met
# Note that when a NaN is present in a column of integers, the column (Series) is casted to float
print("Are df and dfc the same object?", np.may_share_memory(df, dfc))
dfc

In [ ]:
# Filter by a boolean condition on the values of a single column
dfc[dfc['B'] < 20.]

### Assignement

Assignment is typically performed after selection:

In [ ]:
# Make sure to copy the DataFrame if you plan to modify it, and you don't want to change the original object
dfa = df.copy()

# setting values by label (same as by position)
dfa.at[dates[0], 'A'] = -1

# setting and assigning a numpy array
dfa.loc[:, 'D'] = np.array([5] * len(dfa))

# defining a new column
dfa['E'] = np.arange(len(dfa))*0.5

# defining a brend new column by means of a pd.Series: indexes must be the same!
dfa['E prime'] = pd.Series(np.arange(len(dfa))*2, index=dfa.index)

# using masks for assigment
dfa[dfa >= 30] = -dfa

dfa

### Application of a function

User-defined or standard functions can be applied on entire DataFrames or columns, with very short execution times:

In [ ]:
def dcos(theta):
    theta = theta*(np.pi/180)
    return np.cos(theta)
 
dfa['cosine'] = dfa["E"].apply(dcos)
dfa

### Dropping

Dropping columns is an example of a method that does not modify the original object, and returns a new modified object. In other words, if you want to keep the modified DataFrame, perform a new assignment:

```python
df = df.drop(....)
```
Alternatively, the modification of the original object can be forced by specifying `inplace=True` among the arguments.

In [ ]:
dfb = dfa.copy()

# Dropping by column
dfb.drop(['E prime'], axis=1)

#which is equivalent to
dfb = dfb.drop(columns=['E prime'])
#dfb.drop(columns=['E prime'], inplace=True)

dfb

## Missing data

Pandas primarily uses the value `np.nan` to represent missing data. It is by default not included in computations. If there is a NaN entry in a Series of integers, the type of the Series will be changed to floats.

In [ ]:
df_wNan = dfb[dfb > 0]
df_wNan

In [ ]:
# dropping rows with at least a Nan
df_wNan.dropna(how='any')

In [ ]:
# getting a mask
df_wNan.isna()
#df_wNan.notna()

In [ ]:
# filling missing data (not recommended, unless you really mean it)
df_wNan.fillna(value=0)

## Operations

Operations on the elements of a DataFrame are quite straightforward, as the syntax is the same as the one used for Series. Also for DataFrames, operations are performed between elements that share the same labels. Operations on columns are extremly fast, almost as fast as the actual operation between elements in a row.

In [ ]:
# Some statistics (mean() just as an example)
# on rows
print(df.mean(axis=0), '\n')
# on columns
print(df.mean(axis=1), '\n')

In [ ]:
# global operations on columns
df.apply(np.sum) # or whatever function defined by the user

In [ ]:
# Also lambda functions
df.apply(lambda x: x.max() - x.min())

In [ ]:
# syntax is as usual similar to that of numpy arrays
df['S'] = df['A'] + df['C']
df

## Merge

Pandas provides various functions for easily combining together Series and DataFrames in join / merge-type operations.

### Concat

concatenation (adding rows) is straightforward:

In [ ]:
rdf = pd.DataFrame(np.arange(40).reshape(10, 4))
rdf

In [ ]:
# split DataFrame into 3 pieces, row-wise
pieces = [rdf[:3], rdf[3:7], rdf[7:]]
pieces

In [ ]:
# put it back together
pd.concat(pieces)

# in this case, indices are already set; if they are not, indices can be ignored
#pd.concat(pieces, ignore_index=True)

In case of dimension mismatch, Nan are added where needed.

Appending rows and columns also works:

In [ ]:
# appending a single row (as a Series)
s = rdf.iloc[3]
rdf = rdf.append(s, ignore_index=True) # remember to assign the returned object, or use inplace=True
rdf

### Merge/Join

SQL like operations on table can be performed on DataFrames. This is a quite advanced use case, refer to the [doc](https://pandas.pydata.org/pandas-docs/stable/merging.html#merging) for more info/examples.

In [ ]:
left = pd.DataFrame({'key': ['foo', 'bar'], 'lval': [1, 2]})
right = pd.DataFrame({'key': ['foo', 'bar'], 'rval': [4, 5]})

pd.merge(left, right, on="key")

## Grouping

In real world applications, it's quite common that several entries (row) belong to a certain entity, or "group". DataFrames have a powerful tool to perform operations on entries of the same group. The method is called `.groupby()`, and it usually involves one or more of the following steps:

* Splitting the data into groups based on some criteria
* Applying a function to each group independently
* Combining the results into a data structure


In [ ]:
gdf = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                          'foo', 'bar', 'foo', 'foo'],
                    'B' : ['one', 'one', 'two', 'three',
                           'two', 'two', 'one', 'three'],
                    'C' : np.arange(8),
                    'D' : np.linspace(10, -10, 8)})
gdf

In [ ]:
# Grouping and then applying the sum() 
# function to the resulting groups (effective only where numerical values are present)
gdf.groupby('A').sum()

In [ ]:
# Example: find maximum value in column D for each group, and assign the value to a new column
gdf['M'] = gdf.groupby('A')['D'].transform(np.max)
gdf

## Multi-indexing

Hierarchical / Multi-level indexing allows sophisticated data analysis on higher dimensional data. In practice, it enables you to store and manipulate data with an arbitrary number of dimensions in lower dimensional data structures like Series (1D) and DataFrames (2D).

In [ ]:
# Creat multi-dimensional index
tuples = list(zip(['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
          ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']))
multi_index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])
print(multi_index, '\n', type(multi_index), '\n')

# Create multi-indexed dataframe or series
s = pd.Series(np.arange(8)/np.pi, index=multi_index)
s

In [ ]:
# multi-indexing enables further features of the groupby method,
# e.g. when group-by by multiple columns
gdf.groupby(['A','B']).sum()

## Summary: a demonstration of the efficiency of the DataFrame

Let's go the hard way and load in memory a (relatively) large dataset

In [ ]:
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

file_name = "./data/data_000637.txt"
data = pd.read_csv(file_name)
data

Let's now do some operations among (elements of) columns

In [ ]:
itime = dt.datetime.now()
print("Begin time:", itime)

# the one-liner command
data['TIMENS'] = data['TDC_MEAS'] * 25 / 30 + data['BX_COUNTER'] * 25

ftime = dt.datetime.now()
print("End time:", ftime)
print("Elapsed time:", ftime - itime)

data

In [ ]:
# the loop
def conversion(data):
    result = []
    for i in range(len(data)): 
        result.append(data.loc[data.index[i], 'TDC_MEAS'] * 25 / 30. + data.loc[data.index[i], 'BX_COUNTER'] * 25)
    return result

itime = dt.datetime.now()
print("Begin time:", itime)
data['TIMENS'] = conversion(data)
ftime = dt.datetime.now()
print("End time:", ftime)
print("Elapsed time:", ftime - itime)

data